In [1]:
import gym
import torch
import pandas as pd
from sb3_contrib import TRPO, RecurrentPPO, TQC
import numpy as np
import random

In [2]:
class ClipReward(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.best_y = 6
        self.score = 0
        self.last_y = 6

    def reward(self, reward):
        ram = env.unwrapped.ale.getRAM()
        reward = 0
        '''
        reward -= (1/ram[14]) * 40



        if ram[103] > self.score:
            self.score = ram[103]
            reward += 1000
        if ram[14] > self.best_y:
            self.best_y = ram[14]
            reward += 3
        if self.last_y > ram[14]:
            reward -= ram[14]'''
        if 90 <= ram[106] <= 100:
            reward -= 100
        if 140 <= ram[106] <= 141:
            reward += 10000
        reward -= 100/ram[14]
        reward += ram[14]
        if ram[14] >= 172:
            self.best_y = 6
        return reward

In [3]:
def get_action_sample():
    x = random.randint(0, 101)
    if x < 90:
        return 1
    if x < 97:
        return 2
    return 0

In [32]:
env = ClipReward(gym.make("ALE/Freeway-v5", obs_type="ram", render_mode="human", difficulty=1, mode=7))
obs = env.reset()

for _ in range(1000):
    action = get_action_sample()
    obs, reward, done, info = env.step(action)
    #env.render()
    #time.sleep(0.00001)
    print(reward)
    if done:
        obs = env.reset()

UnregisteredEnv: No registered env with id: ALE/Freeway-v5

In [8]:
ITERATIONS = 2000
STEPS_PER_ITERATION = 2048

env = ClipReward(gym.make("ALE/Freeway-v5", obs_type="ram", difficulty=1, mode=3))
obs = env.reset()

model = TRPO("MlpPolicy", env, gamma=0.95, verbose=1, tensorboard_log="tensorlog")
#model = TRPO.load("trpo_1.zip", env=env)
model.learn(total_timesteps=ITERATIONS*STEPS_PER_ITERATION, log_interval=1)
model.save("trpo_1.zip")

model = TRPO.load("trpo_1.zip")

env = ClipReward(gym.make("ALE/Freeway-v5", render_mode="human", obs_type="ram", difficulty=1, mode=3))
obs = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    #env.render()
    #time.sleep(0.00001)
    print(reward)
    if done:
        obs = env.reset()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to tensorlog\TRPO_24


Output()

-----------------------------
| time/              |      |
|    fps             | 246  |
|    iterations      | 1    |
|    time_elapsed    | 8    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| rollout/                  |           |
|    ep_len_mean            | 2.05e+03  |
|    ep_rew_mean            | -1.22e+04 |
| time/                     |           |
|    fps                    | 243       |
|    iterations             | 2         |
|    time_elapsed           | 16        |
|    total_timesteps        | 4096      |
| train/                    |           |
|    explained_variance     | -0.000568 |
|    is_line_search_success | 1         |
|    kl_divergence_loss     | 0.0056    |
|    learning_rate          | 0.001     |
|    n_updates              | 1         |
|    policy_objective       | 0.0189    |
|    value_loss             | 1.22e+04  |
-----------------------------------------


KeyboardInterrupt: 

In [14]:
model = TRPO.load("trpo_1.zip")

env = ClipReward(gym.make("ALE/Freeway-v5", render_mode="human", obs_type="ram", difficulty=1, mode=7))
obs = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    #env.render()
    #time.sleep(0.00001)
    print(reward)
    if done:
        obs = env.reset()

0.0
6.857142857142857
0.0
0.0
-10.666666666666668
0.0
6.857142857142857
6.857142857142857
0.0
6.857142857142857
12.444444444444445
12.444444444444445
17.454545454545453
22.153846153846153
26.666666666666668
31.058823529411764
35.368421052631575
39.61904761904762
43.82608695652174
48.0
52.148148148148145
56.275862068965516
60.38709677419355
64.48484848484848
68.57142857142857
72.64864864864865
76.71794871794872
80.78048780487805
84.83720930232558
88.88888888888889
92.93617021276596
96.9795918367347
101.01960784313725
105.05660377358491
109.0909090909091
113.12280701754386
117.15254237288136
121.18032786885246
125.2063492063492
129.23076923076923
133.2537313432836
137.2753623188406
141.29577464788733
145.31506849315068
149.33333333333334
153.35064935064935
157.36708860759492
161.3827160493827
165.3975903614458
169.41176470588235
173.42528735632183
-10.666666666666668
9989.333333333334
-10.666666666666668
-10.666666666666668
-10.666666666666668
-10.666666666666668
-10.666666666666668
-10.

KeyboardInterrupt: 

In [ ]:
model = TRPO.load("trpo_3.zip")

env = ClipReward(gym.make("ALE/Freeway-v5", render_mode="human", obs_type="ram", difficulty=1, mode=0))
obs = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    #env.render()
    #time.sleep(0.00001)
    print(reward)
    if done:
        obs = env.reset()

In [24]:
import gym
from gym.wrappers import Monitor

env = Monitor(ClipReward(gym.make("ALE/Freeway-v5", render_mode="human", obs_type="ram", difficulty=1, mode=0)), './video', force=True)
state = env.reset()

model = TRPO.load("trpo_1.zip", env=env)

done = False
for i in range(1000):
    action, _states = model.predict(state, deterministic=True)
    state, reward, done, info = env.step(action)
env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
